## Import required modules 

In [1]:
#Required packages
import pandas as pd
import numpy as np 
import pandas as pd 
import re
import string
import nltk 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Merge vertically the three available datasets 
# f1 = pd.read_csv('coronavirus-tweet-id-2020-04-23-00.csv')
# f2 = pd.read_csv('coronavirus-tweet-id-2020-04-23-01.csv')
# f3 = pd.read_csv('coronavirus-tweet-id-2020-04-23-02.csv')
# merged = pd.concat([f1, f2, f3])
# merged.to_csv('merged.csv', index=None, header=True)

## Read the data (April23)

In [3]:
#Import the dataset (Aprinl 23nd)
df = pd.read_csv("April23.csv")
print('the size of the original dataset=', df.shape)

#Remove unnecessary columns
data = df.drop(['place','possibly_sensitive','user_created_at','user_default_profile_image', 
                'user_favourites_count','user_listed_count','user_name', 'user_time_zone', 'user_urls'], axis = 'columns')
data.head()

the size of the original dataset= (849955, 34)


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,text,tweet_url,user_screen_name,user_description,user_followers_count,user_friends_count,user_location,user_screen_name.1,user_statuses_count,user_verified
0,NaN,Thu Apr 23 00:00:00 +0000 2020,NaN,NaN,https://www.spokesman.com/stories/2020/apr/21/...,7,1253111343638609922,NaN,NaN,NaN,...,Colleges and universities in Washington and Id...,https://twitter.com/SpokesmanReview/status/125...,SpokesmanReview,"The region’s only daily newspaper, covering ne...",59594,1495,"Spokane, WA",SpokesmanReview,59663,False
1,NaN,Thu Apr 23 00:00:01 +0000 2020,EarthDay FlattenTheCurve COVID19 NYPHeroes Sol...,https://twitter.com/nyphospital/status/1253111...,NaN,53,1253111347614752769,NaN,NaN,NaN,...,"This #EarthDay, we strive to do better for our...",https://twitter.com/nyphospital/status/1253111...,nyphospital,Proud to be the #4 hospital in the nation and ...,48853,4489,"New York, NY",nyphospital,24947,True
2,NaN,Thu Apr 23 00:00:00 +0000 2020,CODVID19 coronovirus scifi amazonbooks,https://twitter.com/MediaTLF/status/1253111342...,https://amazon.com/PHANTOM-APOCALYPSE-Dystopia...,0,1253111342921207808,NaN,NaN,NaN,...,Written before #CODVID19 #coronovirus happened...,https://twitter.com/MediaTLF/status/1253111342...,MediaTLF,A media company that publishes and distributes...,13982,13942,"New York, USA",MediaTLF,12131,False
3,NaN,Thu Apr 23 00:00:00 +0000 2020,LigaMX,NaN,https://chivaspasion.bolavip.com/ligamx/Diario...,2,1253111343693135879,NaN,NaN,NaN,...,🐐🔥 Solo faltaría una aprobación para que la #L...,https://twitter.com/RebanoPasion/status/125311...,RebanoPasion,Sitio no oficial del Guadalajara creado por y ...,38890,56,Guadalajara,RebanoPasion,33317,True
4,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,NaN,0,1253111349040656384,NaN,NaN,NaN,...,RT @StephenNP: Also look first into the nares ...,https://twitter.com/KirklandWalsh/status/12531...,KirklandWalsh,"PhD, FNP-c, GNP-c, specializing in wounds, doc...",723,2172,NaN,KirklandWalsh,95748,False


In [4]:
#Print the counts of verified and unverified accounts
data['user_verified'].value_counts()

False    828678
True      21277
Name: user_verified, dtype: int64

## Remove verified Twitter accounts

In [5]:
#Remove verified Twitter accounts
data_unverified = data.drop(data[data['user_verified']== True].index)
data_unverified.shape

(828678, 25)

In [6]:
#Show the number of English and non-English tweets
data_unverified['lang'].value_counts()

en     518143
es     101616
ja      38302
fr      29414
und     27393
        ...  
sd          7
dv          5
ckb         3
hy          2
ka          1
Name: lang, Length: 62, dtype: int64

## Remove non-English tweets

In [7]:
#Remove non-English tweets
data_unverified_en = data_unverified[data_unverified.lang == 'en']
data_unverified_en.shape

(518143, 25)

## Create two columns of cities and countries/provinces

In [8]:
#Split city and country (province) in user_location column  
new = data_unverified_en.loc[:, "user_location"].str.split(",", n = 1, expand = True) 
  
# making separate city column from new data frame 
data_unverified_en.loc[:,"city"]= new[0] 

  
# making separate country or province column from new data frame 
data_unverified_en.loc[:,"country"]= new[1] 

data_unverified_en

c:\users\maede\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\maede\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_screen_name,user_description,user_followers_count,user_friends_count,user_location,user_screen_name.1,user_statuses_count,user_verified,city,country
0,NaN,Thu Apr 23 00:00:00 +0000 2020,NaN,NaN,https://www.spokesman.com/stories/2020/apr/21/...,7,1253111343638609922,NaN,NaN,NaN,...,SpokesmanReview,"The region’s only daily newspaper, covering ne...",59594,1495,"Spokane, WA",SpokesmanReview,59663,False,Spokane,WA
2,NaN,Thu Apr 23 00:00:00 +0000 2020,CODVID19 coronovirus scifi amazonbooks,https://twitter.com/MediaTLF/status/1253111342...,https://amazon.com/PHANTOM-APOCALYPSE-Dystopia...,0,1253111342921207808,NaN,NaN,NaN,...,MediaTLF,A media company that publishes and distributes...,13982,13942,"New York, USA",MediaTLF,12131,False,New York,USA
4,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,NaN,0,1253111349040656384,NaN,NaN,NaN,...,KirklandWalsh,"PhD, FNP-c, GNP-c, specializing in wounds, doc...",723,2172,NaN,KirklandWalsh,95748,False,NaN,NaN
5,NaN,Thu Apr 23 00:00:00 +0000 2020,NaN,https://twitter.com/winknews/status/1253111342...,https://bit.ly/34XCLO5,1,1253111342283649026,NaN,NaN,NaN,...,winknews,Southwest Florida's News Leader. Have a news t...,89493,689,"Fort Myers, Fla.",winknews,95769,False,Fort Myers,Fla.
7,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,NaN,2,1253111348407328768,inminivanhell,1.253084e+18,4.675062e+09,...,kate_barrel,"Books, the outdoors. Community .Gone with the ...",70,97,Location Location,kate_barrel,4307,False,Location Location,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849947,NaN,Thu Apr 23 23:52:23 +0000 2020,NaN,https://twitter.com/doloresabernthy/status/125...,NaN,97,1253471814749208579,NaN,NaN,NaN,...,doloresabernthy,"#NEIL what's happened, happened. which is an e...",1493,512,she/her,doloresabernthy,71350,False,she/her,None
849948,NaN,Thu Apr 23 23:52:22 +0000 2020,NaN,https://twitter.com/hoyeengurl/status/12533203...,NaN,0,1253471810647142400,NaN,NaN,NaN,...,purfilme,I don't know maybe I'm depressed\n\n\n\n\nele/...,196,382,Campinas - São Paulo,purfilme,9922,False,Campinas - São Paulo,None
849951,NaN,Thu Apr 23 23:52:22 +0000 2020,NaN,NaN,NaN,0,1253471810760380418,NaN,NaN,NaN,...,BriTheBrat18,NaN,99,93,"Boston, MA",BriTheBrat18,25984,False,Boston,MA
849952,NaN,Thu Apr 23 23:52:22 +0000 2020,NaN,NaN,NaN,0,1253471811725135873,NaN,NaN,NaN,...,Brandonbrad53,NaN,381,3516,SUFFOLK COUNTY NY,Brandonbrad53,3395,False,SUFFOLK COUNTY NY,None


## Convert cities and countries strings to lowercase

In [9]:
#Convert cities and countries strings to lowercase
data_unverified_en.loc[: , "country"] = data_unverified_en.loc[: , "country"].str.lower()
data_unverified_en.loc[: , "city"] = data_unverified_en.loc[: , "city"].str.lower()

data_unverified_en["country"].value_counts()

c:\users\maede\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


 usa               27989
 india             12113
 england            9101
 ca                 8483
 nigeria            5304
                   ...  
 damn wa               1
-87.684423             1
mn.                    1
 sol                   1
 mombasa,kenya.        1
Name: country, Length: 11856, dtype: int64

In [10]:
#To remove the single space at the beginning of strings in country column
data_unverified_en.loc[: , 'country']= data_unverified_en.loc[: , 'country'].str.lstrip()

### Import the reference data (list_of_municipalities_of_canada-1633j.csv)

In [11]:
#import the 'list_of_cities_Canada' dataset (The reference for cities of Canada)
cities_reference = pd.read_csv("list_of_municipalities_of_canada-1633j.csv")
cities_reference.head()

,SNo,Provinces,Name,Municipal sub-type,Population -2011,Population -2006,Change (%),Land area (km²),Population density (per km2)
0,1.0,Ontario,Addington Highlands,Township,"2,532","2,512",0.8,"1,329.93",1.9
1,2.0,Ontario,Adelaide-Metcalfe,Township,"3,028","3,135",?3.4,331.38,9.1
2,3.0,Ontario,Adjala-Tosorontio,Township,"10,603","10,695",?0.9,372.39,28.5
3,4.0,Ontario,Admaston/Bromley,Township,"2,844","2,716",4.7,524.6,5.4
4,5.0,Ontario,Ajax,Town,"1,09,600","90,167",21.6,67.07,"1,634.20"


## Preprocess the reference data 

In [12]:
#Convert the city names and provinces of Canada to lowercase in the reference data
cities_reference.Provinces= cities_reference.Provinces.str.lower()
cities_reference.Name= cities_reference.Name.str.lower()

#To remove the single space at the beginning of strings in country column in reference data
cities_reference['Provinces']= cities_reference['Provinces'].str.lstrip()

#Make a numpy array from the list of canadian cities/provinces 
Canadian_cities = cities_reference.Name.to_numpy()
Canadian_provinces = cities_reference.Provinces.to_numpy()

#Remove duplicates from the provinces list
Canadian_provinces = list(dict.fromkeys(Canadian_provinces))

#Remove single space from the provinces list
Canadian_provinces.remove("")

#Remove nan from Canadian_provinces and Canadian_cities
Canadian_provinces = list(filter(lambda x: str(x) != 'nan', Canadian_provinces))
Canadian_cities = list(filter(lambda x: str(x) != 'nan', Canadian_cities))

Canadian_provinces.append('newfoundland')
Canadian_provinces

['ontario',
 'nova scotia',
 'newfoundland and labrador',
 'alberta',
 'british columbia',
 'manitoba',
 'new brunswick',
 'edward island',
 'quebec',
 'saskatchewan',
 'newfoundland']

In [13]:
#make a list of all Canadian cities and provinces
all_reference = []
all_reference.extend(Canadian_provinces)
all_reference.extend(Canadian_cities)
all_reference


['ontario',
 'nova scotia',
 'newfoundland and labrador',
 'alberta',
 'british columbia',
 'manitoba',
 'new brunswick',
 'edward island',
 'quebec',
 'saskatchewan',
 'newfoundland',
 'addington highlands',
 'adelaide-metcalfe',
 'adjala-tosorontio',
 'admaston/bromley',
 'ajax',
 'alberton',
 'alfred and plantagenet',
 'algonquin highlands',
 'alnwick/haldimand',
 'amaranth',
 'amherstburg',
 'the archipelago',
 'armour',
 'armstrong',
 'arnprior',
 'arran-elderslie',
 'ashfield-colborne-wawanosh',
 'asphodel-norwood',
 'assiginack',
 'athens',
 'atikokan',
 'augusta',
 'aurora',
 'aylmer',
 'baldwin',
 'bancroft',
 'barrie',
 'bayham',
 'beckwith',
 'belleville',
 'billings',
 'black river-matheson',
 'blandford-blenheim',
 'blind river',
 'the blue mountains',
 'bluewater',
 'bonfield',
 'bonnechere valley',
 'bracebridge',
 'bradford west gwillimbury',
 'brampton',
 'brant',
 'brantford',
 'brethour',
 'brighton',
 'brock',
 'brockton',
 'brockville',
 'brooke-alvinston',
 'bruce

## Select Canadian tweets

In [14]:
# select the rows from 'data_unverified_en' dataset with country=='canada' or country ∈ 'Canadian_provinces' list, 
# or the rows with city=='canada' or city ∈ 'Canadian_provinces' list,
data_unverified_en_1 = data_unverified_en[data_unverified_en['country'].isin(['canada']) 
                                           |(data_unverified_en.city == 'canada')
                                           |data_unverified_en['city'].isin(Canadian_provinces)
                                           |data_unverified_en['country'].isin(Canadian_provinces)]

# select the rows from 'data_unverified_en' dataset with country=='ca'  
data_unverified_en_2 = data_unverified_en[data_unverified_en['country'].isin(['ca'])] 

# select the rows in 'data_unverified_en_2' dataset that their city column's values exist in 
# Canadian_cities list (we do this to make sure that 'ca' indicates 'Canada' not for example 'California' )

data_unverified_en_ca = data_unverified_en_2[data_unverified_en_2['city'].isin(Canadian_cities)]                                          

In [15]:
#Merge datasets to have a whole dataset of Canadian tweets 
data_unverified_en_canada = data_unverified_en_1.append(data_unverified_en_ca)

#Our final dataset
data_unverified_en_canada

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_screen_name,user_description,user_followers_count,user_friends_count,user_location,user_screen_name.1,user_statuses_count,user_verified,city,country
68,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,NaN,0,1253111348151517185,NaN,NaN,NaN,...,NinjaRapheal,"Proud Canadian, Hockey fan, Toronto Blue Jays ...",1426,1124,"East Kootenay, British Columbia",NinjaRapheal,142604,False,east kootenay,british columbia
79,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,https://twitter.com/Reuters/status/12526246774...,0,1253111348780830723,DrTedros,NaN,189868631.0,...,YoungdoungT,NaN,18,137,"Toronto, Ontario",YoungdoungT,193,False,toronto,ontario
293,NaN,Thu Apr 23 00:00:06 +0000 2020,NaN,NaN,NaN,0,1253111369597095936,NaN,NaN,NaN,...,NanananaBrown,Loves the BlueJays but love my daughter's and ...,1006,982,"Toronto, Ontario",NanananaBrown,109276,False,toronto,ontario
319,NaN,Thu Apr 23 00:00:06 +0000 2020,NaN,NaN,NaN,0,1253111369072869381,NaN,NaN,NaN,...,runcoachphil,"Running Room Regional Manager, Parliament/ Arm...",1023,87,"Ottawa, Canada",runcoachphil,38315,False,ottawa,canada
326,NaN,Thu Apr 23 00:00:06 +0000 2020,NaN,NaN,https://www.gq.com/story/las-vegas-polio-coron...,0,1253111369618030593,NaN,NaN,NaN,...,ExtremeDeals81,Discover The Greatest Online Shopping Daily De...,1961,1723,"toronto, ontario",ExtremeDeals81,20090,False,toronto,ontario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831895,NaN,Thu Apr 23 23:11:26 +0000 2020,NaN,NaN,NaN,0,1253461509860016129,NaN,NaN,NaN,...,dmunk45,NaN,134,286,"Danville, CA",dmunk45,7964,False,danville,ca
839122,NaN,Thu Apr 23 23:31:11 +0000 2020,NaN,NaN,NaN,0,1253466480320016384,NaN,NaN,NaN,...,wchrisb57,"Calvin Coolidge conservative...Love baseball, ...",411,794,"Chatsworth,CA",wchrisb57,14495,False,chatsworth,ca
843044,NaN,Thu Apr 23 23:32:48 +0000 2020,COVID19,NaN,NaN,0,1253466887754665984,NaN,NaN,NaN,...,mattmiszewski,"Husband, Father, Executive, CIO, GM, SVP, Boar...",1607,2141,"Danville, CA",mattmiszewski,10004,False,danville,ca
845774,NaN,Thu Apr 23 23:33:54 +0000 2020,NaN,NaN,NaN,1,1253467163173609472,rashauna_alyse,1.253396e+18,139613138.0,...,theekbradshaw,"Product of Compton & the CSU. Single, Black fe...",428,560,"Compton, CA",theekbradshaw,36526,False,compton,ca


## Separate the retweets from the original tweets into two separate dataframes

In [16]:
data_unverified_en_canada.loc[data_unverified_en_canada['text'].str.contains('RT @'), 'retweet'] = "true"

data_unverified_en_canada['retweet'] = data_unverified_en_canada['retweet'].fillna("false")

data_unverified_en_canada.retweet.value_counts()

true     5145
false    2303
Name: retweet, dtype: int64

In [17]:
#Separate the retweets from the original tweets into two separate files
data_unverified_en_canada_original = data_unverified_en_canada[data_unverified_en_canada['retweet']== "false"]
data_unverified_en_canada_retweet = data_unverified_en_canada[data_unverified_en_canada['retweet']== "true"]
print("the size of the dataset with retweets=", data_unverified_en_canada_retweet.shape)
print("the size of the dataset with original tweets=", data_unverified_en_canada_original.shape)

the size of the dataset with retweets= (5145, 28)
the size of the dataset with original tweets= (2303, 28)


## Export the final data

In [18]:
#Export the final dataset (data_unverified_en_canada) with unverified English tweets from Canada
data_unverified_en_canada.to_csv('data_unverified_en_canada.csv', index=None, header=True)

## Prepare the data for visualization

In [19]:
# get a Series containing counts of cities in 'data_unverified_en_canada' dataframe
value_counts = data_unverified_en_canada['city'].value_counts(dropna=True, sort=True)
type(value_counts)

pandas.core.series.Series

In [20]:
#Convert Pandas Series to a new dataframe
city_val_counts = pd.DataFrame(value_counts)
city_val_counts

,city
canada,1724
toronto,1091
ontario,663
vancouver,310
ottawa,295
...,...
tillsonburg,1
o-town,1
moose jaw no. 161,1
eastern townships,1


In [21]:
#change the index to the column of cities in 'city_val_counts' dataframe
city_value_counts = city_val_counts.reset_index()
city_value_counts.columns = ['location', 'counts']
city_value_counts

,location,counts
0,canada,1724
1,toronto,1091
2,ontario,663
3,vancouver,310
4,ottawa,295
...,...,...
496,tillsonburg,1
497,o-town,1
498,moose jaw no. 161,1
499,eastern townships,1


In [22]:
#Export the locations along with tweets came from each location. 
city_value_counts.to_csv('location_value_counts.csv', index=None, header=True)